In [1]:
import pandas as pd
import os
import numpy as np
os.chdir("D:/data")
data_reg = pd.read_csv('fred_quarterly v3.csv')
data_reg['Date'] = pd.to_datetime(data_reg['month'])
data_reg.head()

,month,quarter,Target,gnp,gdpdef,gdp,Date
0,2000-01,2000Q1,12935.252,10035.263,77.325,10002.179,2000-01-01
1,2000-04,2000Q2,13170.749,10283.391,77.807,10247.720,2000-04-01
2,2000-07,2000Q3,13183.890,10348.645,78.263,10318.165,2000-07-01
3,2000-10,2000Q4,13262.250,10490.430,78.688,10435.744,2000-10-01
4,2001-01,2001Q1,13219.251,10510.297,79.204,10470.231,2001-01-01


In [2]:
data_reg['Month'] = [i.month for i in data_reg['Date']]
data_reg['Year'] = [i.year for i in data_reg['Date']]
data_reg['Series'] = np.arange(1,len(data_reg)+1)
data_reg = data_reg[['Series', 'Year', 'Month', 'Target']] 
data_reg.head()

,Series,Year,Month,Target
0,1,2000,1,12935.252
1,2,2000,4,13170.749
2,3,2000,7,13183.890
3,4,2000,10,13262.250
4,5,2001,1,13219.251


In [3]:
train = data_reg[data_reg['Year'] < 2018]
test = data_reg[data_reg['Year'] >= 2018]
train.shape, test.shape

((72, 4), (13, 4))

In [4]:
from pycaret.regression import *
s = setup(data = train, test_data = test, target = 'Target', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,Target
2,Original Data,"(72, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(72, 5)"


In [5]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,508.0602,557702.0381,577.5195,-1.7690,0.0347,0.0297,0.9500


In [6]:
prediction_holdout=predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,714.1054,569143.5935,754.4161,-1.5555,0.0409,0.0381


In [21]:
predictions = predict_model(best, data=data_reg)
predictions['Date'] = pd.date_range(start='2000-01-01', end = '2021-04-01',freq='3M')

In [22]:
final_best = finalize_model(best)

In [26]:
future_dates = pd.date_range(start = '2021-04-01', end = '2022-07-01', freq = '3M')
future_df = pd.DataFrame()
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(145,(145+len(future_dates)))
future_df.head()

,Month,Year,Series
0,4,2021,145
1,7,2021,146
2,10,2021,147
3,1,2022,148
4,4,2022,149


In [27]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

,Month,Year,Series,Label
0,4,2021,145,17981.328602
1,7,2021,146,18198.211462
2,10,2021,147,18287.472949
3,1,2022,148,17978.518048
4,4,2022,149,17981.328602
